This notebook contains reproducible code to calculate errors reported in the NLDL'24 report

In [35]:
import numpy as np
import os
import re
from collections import defaultdict

In [36]:
workdir = !pwd
workdir = workdir[0]
workdir

'/mnt/c/Arbeid/Github_Repo/NLDL_report/data_imputation_algs/TestingFramework/bin/Debug/Results'

In [37]:
scenarios = ['miss_perc']
dataset = 'oceantidepressure'

In [38]:
def config_error_calc(steps_arrange, list_algs, sce_dir):
    lengths = np.arange(start=steps_arrange[0], stop=steps_arrange[1], step=steps_arrange[2])
    global_path = sce_dir + '/error/'

    output_mse = global_path + "mse/MSE_"
    output_mae = global_path + "mae/MAE_"
    output_cor = global_path + "correlation.dat"
    input_missingmat = sce_dir + '/' + "recovery/values/recovered_matrices/recoveredMat"

    list_algos = list_algs
    
    return lengths, output_mse, output_rmse, output_mae, output_cor, input_missingmat, list_algos


def mse(lengths, output_mse, output_rmse, output_mae, output_cor, input_missingmat, list_algos):
    err = defaultdict(list)
    for length in lengths:
        dftest = np.loadtxt(input_missingmat + str(length) + ".txt", dtype=float)

        for length in lengths:
            df = np.loadtxt(input_missingmat + str(length) + ".txt", dtype=float)
            df = df[~np.isnan(df).any(axis=1)]
            ref = df[:, 0]        

            for i in range(1, df.shape[1]):
                #fileName = output_mse + list_algos[i - 1] + ".dat"
                comp = df[:, i]
                comp -= ref
                msqe_val = np.mean(comp ** 2)
                err[list_algos[i-1]].append(msqe_val)
    
    for alg in err.keys():
        errors = err[alg]
        print('MSE:', alg, ':', sum(errors) / len(errors))
        
def mae(lengths, output_mse, output_rmse, output_mae, output_cor, input_missingmat, list_algos):
    err = defaultdict(list)
    for length in lengths:
        dftest = np.loadtxt(input_missingmat + str(length) + ".txt", dtype=float)

        for length in lengths:
            df = np.loadtxt(input_missingmat + str(length) + ".txt", dtype=float)
            df = df[~np.isnan(df).any(axis=1)]
            ref = df[:, 0]        

            for i in range(1, df.shape[1]):
                #fileName = output_mse + list_algos[i - 1] + ".dat"
                comp = abs(df[:, i] - ref)
                mae_val = np.mean(comp)
                err[list_algos[i-1]].append(mae_val)
    
    for alg in err.keys():
        errors = err[alg]
        print('MAE:', alg, ':', sum(errors) / len(errors))



In [39]:
for scenario in scenarios:
    try:
        #print('------------------------------')
        #print(scenario)
        error_calculation_r_filepath = workdir + '/' + scenario + '/' + dataset + '/scripts/precision/error_calculation.r'
        with open(error_calculation_r_filepath, 'r') as r_file:
            for _, line in enumerate(r_file, 1):
                if 'lengths <- seq.int' in line:
                    steps_arange = re.findall(r'\d+', line)
                    steps_arange = [int(step_arange) for step_arange in steps_arange]
                elif 'list_algos <- c' in line:
                    list_algs = re.findall(r'"([^"]*)"', line)

        lengths, output_mse, output_rmse, output_mae, output_cor, input_missingmat, list_algos = \
            config_error_calc(steps_arange, list_algs, workdir + '/' + scenario + '/' + dataset )
        mse(lengths, output_mse, output_rmse, output_mae, output_cor, input_missingmat, list_algos)
        mae(lengths, output_mse, output_rmse, output_mae, output_cor, input_missingmat, list_algos)
    except:
        pass
    #break

MSE: brits : 0.26543467231341183
MSE: m-rnn : 0.30186371840828063
MSE: csdipy : 0.6916478820820502
MAE: brits : 0.2601759538859269
MAE: m-rnn : 0.4090353522252748
MAE: csdipy : 0.7870813573851123
